In [ ]:
# Random instance generator for the TSP problem


import networkx as nx


def generate_instance(n=200) -> tuple[nx.Graph, list[tuple[int, int]]]:
    # generate n random points
    import random

    points = [(random.randint(0, 1000), random.randint(0, 1000)) for _ in range(n)]

    def eucl_dist(p1, p2):
        return round(((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5)

    # create weighted graph
    G = nx.Graph()
    for i in range(n):
        for j in range(i + 1, n):
            G.add_edge(i, j, weight=eucl_dist(points[i], points[j]))
    return G, points

In [ ]:
def draw_integral_solution(solution: nx.Graph, points: list[tuple[int, int]]):
    import matplotlib.pyplot as plt

    plt.figure()
    plt.title("Integral Solution")
    # equal aspect ratio
    plt.gca().set_aspect("equal", adjustable="box")
    pos = {i: points[i] for i in range(len(points))}
    nx.draw(
        solution,
        pos,
        with_labels=False,
        node_size=10,
        node_color="blue",
        edge_color="black",
    )
    plt.show()


def draw_fractional_solution(solution: nx.Graph, points: list[tuple[int, int]]):
    import matplotlib.pyplot as plt

    plt.figure()
    plt.title("Linear Relaxation")
    # equal aspect ratio
    plt.gca().set_aspect("equal", adjustable="box")
    pos = {i: points[i] for i in range(len(points))}
    # draw fractional edges in red. An edge is fractional if the attribute `x` is between 0.01 and 0.99
    for u, v, d in solution.edges(data=True):
        if 0.01 < d["x"] < 0.99:
            nx.draw(
                nx.Graph([(u, v)]),
                pos,
                with_labels=False,
                node_size=10,
                node_color="red",
                edge_color="red",
            )
    # draw integral edges in black. An edge is integral if the attribute `x` is > 0.99
    for u, v, d in solution.edges(data=True):
        if d["x"] > 0.99:
            nx.draw(
                nx.Graph([(u, v)]),
                pos,
                with_labels=False,
                node_size=10,
                node_color="blue",
                edge_color="black",
            )
    plt.show()


def draw_overlap(
    solution: nx.Graph, linear_relaxation: nx.Graph, points: list[tuple[int, int]]
):
    # draw the edges of the integral solution that have a value >= 0.5 in the fractional solution
    import matplotlib.pyplot as plt

    plt.figure()
    plt.title("Overlap")
    # equal aspect ratio
    plt.gca().set_aspect("equal", adjustable="box")
    # draw all nodes
    nx.draw(
        solution,
        points,
        with_labels=False,
        node_size=10,
        node_color="blue",
        edge_color="grey",
        width=0.1,
    )
    # draw the edges of the integral solution that have a value >= 0.5 in the fractional solution
    n = 0
    for u, v, d in solution.edges(data=True):
        # if not in linear relaxation, skip
        if not linear_relaxation.has_edge(u, v):
            continue
        if linear_relaxation[u][v]["x"] >= 0.5:
            nx.draw(
                nx.Graph([(u, v)]),
                points,
                with_labels=False,
                node_size=10,
                node_color="blue",
                edge_color="green",
            )
            n += 1

    plt.show()
    return n

In [ ]:
from solution_dantzig import GurobiTspSolver
from solution_relaxation import GurobiTspRelaxationSolver

samples = []
n = 200


def get_sample():
    instance, points = generate_instance(n)

    print("Solving the linear relaxation")
    relaxation_solver = GurobiTspRelaxationSolver(instance)
    relaxation_solver.solve()
    relaxed_solution = relaxation_solver.get_solution()
    linear_relaxation_value = relaxation_solver.get_objective()
    assert relaxed_solution is not None
    draw_fractional_solution(relaxed_solution, points)

    print("Solving the integral problem")
    solver = GurobiTspSolver(instance)
    solver.solve(30)
    solution = solver.get_solution()
    objective_value = solver.get_objective()
    assert solution is not None
    draw_integral_solution(solution, points)
    overlap = draw_overlap(solution, relaxed_solution, points)
    print(
        f"Objective value: {objective_value}, Linear relaxation value: {linear_relaxation_value}, Overlap: {overlap}/{n}"
    )
    samples.append((objective_value, linear_relaxation_value, overlap))

In [ ]:
# Run me to get a sample on linear relaxation and integral solution
get_sample()
print(f"There are now {len(samples)} samples")

In [ ]:
for i, (objective_value, linear_relaxation_value, overlap) in enumerate(samples):
    print(
        f"Sample {i}: Objective value: {objective_value}, Linear relaxation value: {linear_relaxation_value}, Overlap: {overlap}/{n}"
    )